# Huấn luyện AI chơi cờ Gomoku (Caro) với Deep Reinforcement Learning

Notebook này cho phép bạn huấn luyện và chạy AI cờ Gomuko sử dụng Deep Reinforcement Learning trên Google Colab.

## Thiết lập môi trường

Đầu tiên, chúng ta cần thiết lập các gói cần thiết và tải mã nguồn từ GitHub.

In [ ]:
# Cài đặt các gói cần thiết
!pip install pygame numpy torch tensorboard tqdm matplotlib

# Kiểm tra CUDA
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## Kết nối với Google Drive

Kết nối với Google Drive để lưu trữ các mô hình và kết quả huấn luyện. Điều này giúp bạn không bị mất dữ liệu nếu phiên Colab bị ngắt kết nối.

In [ ]:
# Kết nối với Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục cho dự án trên Google Drive (nếu chưa tồn tại)
import os
DRIVE_PROJECT_DIR = '/content/drive/MyDrive/BotGomuko'
DRIVE_MODELS_DIR = f"{DRIVE_PROJECT_DIR}/saved_models"
DRIVE_LOGS_DIR = f"{DRIVE_PROJECT_DIR}/logs"

for directory in [DRIVE_PROJECT_DIR, DRIVE_MODELS_DIR, DRIVE_LOGS_DIR]:
    os.makedirs(directory, exist_ok=True)

print(f"Google Drive đã được kết nối và thư mục dự án đã sẵn sàng tại {DRIVE_PROJECT_DIR}")

## Clone mã nguồn từ GitHub

Sao chép mã nguồn từ kho GitHub để chạy trên Colab.

In [ ]:
# Clone repository (thay đổi URL nếu cần)
!git clone https://github.com/yourusername/BotGomuko.git
# Hoặc bạn có thể tải lên thủ công các file từ máy tính của bạn

# Di chuyển đến thư mục dự án
%cd BotGomuko

## Hoặc tải lên thủ công

Nếu bạn không có GitHub repository, bạn có thể tải lên các file thủ công. Chạy hàm bên dưới để tạo cấu trúc thư mục cần thiết.

In [ ]:
# Tạo cấu trúc thư mục nếu tải lên thủ công
import os

# Tạo các thư mục cần thiết
directories = ['agents', 'game', 'utils', 'training', 'saved_models', 'logs']
for directory in directories:
    os.makedirs(directory, exist_ok=True)

print("Cấu trúc thư mục đã được tạo. Vui lòng tải lên các file vào thư mục tương ứng.")

## Kiểm tra cấu trúc thư mục

Đảm bảo rằng tất cả các file cần thiết đã có sẵn.

In [ ]:
# Liệt kê các file và kiểm tra cấu trúc thư mục
!ls -R

## Thiết lập pygame cho Colab

Vì Colab không hỗ trợ hiển thị đồ họa trực tiếp, chúng ta cần thiết lập một số bước để pygame hoạt động.

In [ ]:
# Thiết lập pygame headless mode cho Colab
import os
os.environ['SDL_VIDEODRIVER'] = 'dummy'

# Patch cho pygame trong colab
import pygame
pygame.init()
pygame.display.set_mode((1, 1))
print("Pygame đã được thiết lập!")

## Tạo script để tự động lưu mô hình vào Google Drive

Tạo một script wrapper để đảm bảo các mô hình được lưu vào Google Drive thay vì lưu trong Colab (sẽ mất khi phiên hết hạn).

In [ ]:
%%writefile save_to_drive.py
import os
import sys
import time
import shutil
from pathlib import Path

def create_drive_model_path(model_path, drive_base_dir):
    """Tạo đường dẫn tương ứng trên Google Drive"""
    path = Path(model_path)
    drive_path = os.path.join(drive_base_dir, path.name)
    return drive_path

def setup_drive_auto_save(original_args, drive_models_dir, drive_logs_dir):
    """Thiết lập tự động lưu mô hình vào Google Drive"""
    import sys
    import time
    from main import self_train

    # Phân tích tham số
    args = {}
    i = 0
    while i < len(original_args):
        if original_args[i].startswith('--'):
            param = original_args[i][2:]
            if i + 1 < len(original_args) and not original_args[i+1].startswith('--'):
                args[param] = original_args[i+1]
                i += 2
            else:
                args[param] = True
                i += 1
        else:
            i += 1

    # Thiết lập tham số
    total_episodes = int(args.get('episodes', 10000))
    save_interval = int(args.get('save-interval', 100))
    resume = 'resume' in args
    
    # Đường dẫn mô hình
    model_path = args.get('model', 'saved_models/rl_self_train.pt')
    
    # Tạo phiên bản drive của model_path
    drive_model_path = create_drive_model_path(model_path, drive_models_dir)
    
    # Tạo thư mục TensorBoard trên Drive
    timestr = time.strftime('%Y%m%d-%H%M%S')
    drive_log_dir = f"{drive_logs_dir}/self_train_{timestr}"
    os.makedirs(drive_log_dir, exist_ok=True)
    
    print(f"\n=== AUTO-SAVE TO GOOGLE DRIVE ENABLED ===")
    print(f"Model path: {drive_model_path}")
    print(f"TensorBoard logs: {drive_log_dir}")
    
    # Nếu resume và có model trong Drive, sao chép về Colab
    if resume and os.path.exists(drive_model_path):
        print(f"Tìm thấy mô hình trên Drive, sao chép về Colab để tiếp tục huấn luyện...")
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        shutil.copy2(drive_model_path, model_path)
    
    # Hook for model saving
    original_save = self_train.__globals__['RLAgent'].save
    
    def save_hook(self, filepath):
        # Gọi hàm save gốc
        original_save(self, filepath)
        
        # Nếu là checkpoint hoặc mô hình cuối cùng, lưu vào Drive
        if '_ep' in filepath or 'self_train.pt' in filepath:
            drive_path = create_drive_model_path(filepath, drive_models_dir)
            os.makedirs(os.path.dirname(drive_path), exist_ok=True)
            shutil.copy2(filepath, drive_path)
            print(f"Đã sao chép mô hình vào Drive: {drive_path}")
            
            # Nếu có biểu đồ tiến độ, sao chép nó
            progress_file = filepath.replace('.pt', '_progress_final.png')
            if os.path.exists(progress_file):
                drive_progress = create_drive_model_path(progress_file, drive_models_dir)
                shutil.copy2(progress_file, drive_progress)
                print(f"Đã sao chép biểu đồ tiến độ vào Drive: {drive_progress}")
    
    # Thay thế hàm save
    self_train.__globals__['RLAgent'].save = save_hook
    
    # Start training
    self_train(
        total_episodes=total_episodes,
        save_interval=save_interval,
        model_path=model_path,
        resume=resume,
        use_tensorboard=True
    )
    
if __name__ == "__main__":
    if len(sys.argv) < 3:
        print("Usage: python save_to_drive.py <drive_models_dir> <drive_logs_dir> [training args...]")
        sys.exit(1)
        
    drive_models_dir = sys.argv[1]
    drive_logs_dir = sys.argv[2]
    original_args = sys.argv[3:]
    
    setup_drive_auto_save(original_args, drive_models_dir, drive_logs_dir)

## Huấn luyện mô hình với lưu trữ tự động trên Google Drive

Bây giờ chúng ta có thể bắt đầu huấn luyện mô hình với các checkpoints sẽ được lưu tự động vào Google Drive.

In [ ]:
# Chạy huấn luyện với số tập nhỏ, tự động lưu vào Google Drive
!python save_to_drive.py "$DRIVE_MODELS_DIR" "$DRIVE_LOGS_DIR" --episodes 1000 --save-interval 100

## Theo dõi quá trình huấn luyện với TensorBoard

Bạn có thể theo dõi quá trình huấn luyện bằng TensorBoard.

In [ ]:
# Khởi động TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs

## Huấn luyện đầy đủ

Nếu bạn muốn huấn luyện mô hình với nhiều tập hơn (khuyến nghị ít nhất 5000-10000 tập):

In [ ]:
# Huấn luyện với nhiều tập hơn - hãy cẩn thận vì điều này có thể mất nhiều thời gian
# Kết quả sẽ tự động được lưu vào Google Drive
!python save_to_drive.py "$DRIVE_MODELS_DIR" "$DRIVE_LOGS_DIR" --episodes 10000 --save-interval 500

## Kiểm tra tiến trình huấn luyện từ Google Drive

Sau khi huấn luyện, kiểm tra biểu đồ tiến trình và các mô hình đã lưu trên Google Drive.

In [ ]:
# Liệt kê các mô hình đã lưu trên Google Drive
!ls -la "$DRIVE_MODELS_DIR"

# Hiển thị một trong những biểu đồ tiến trình từ Google Drive
from IPython.display import Image
import os

progress_files = [f for f in os.listdir(DRIVE_MODELS_DIR) if f.endswith('_progress_final.png') or f.endswith('_progress_ep*.png')]
if progress_files:
    latest_progress = progress_files[-1]
    Image(filename=f'{DRIVE_MODELS_DIR}/{latest_progress}')
else:
    print("Không tìm thấy biểu đồ tiến trình huấn luyện trên Google Drive")

## Tiếp tục huấn luyện mô hình đã có từ Google Drive

Nếu bạn muốn tiếp tục huấn luyện từ một mô hình đã lưu trước đó trên Google Drive:

In [ ]:
# Tiếp tục huấn luyện từ mô hình đã lưu trên Google Drive
!python save_to_drive.py "$DRIVE_MODELS_DIR" "$DRIVE_LOGS_DIR" --episodes 2000 --save-interval 200 --model saved_models/rl_self_train.pt --resume

## Lưu ý

1. Google Colab có thể ngắt kết nối sau vài giờ, đặc biệt khi sử dụng GPU miễn phí. Huấn luyện một mô hình tốt có thể mất nhiều giờ hoặc thậm chí nhiều ngày.
2. Với tính năng tự động lưu vào Google Drive, bạn không cần lo lắng về việc mất mô hình khi Colab ngắt kết nối.
3. Để tiếp tục huấn luyện, chỉ cần khởi động lại notebook và sử dụng tùy chọn `--resume`.
4. Để có mô hình tốt, cần huấn luyện ít nhất 5000-10000 tập.
5. Bạn có thể điều chỉnh các thông số trong file `config.json` hoặc sử dụng các đối số dòng lệnh để tối ưu hóa quá trình huấn luyện.